# Importing

In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
import random

In [2]:
df= pd.read_csv('cleaned_bank.csv')

In [3]:
try:
    del df['Unnamed: 0']
except:
    pass

In [4]:
df.head()

,age,marital,education,default,housing,loan,contact,month,campaign,pdays,...,emp.var.rate,cons.price.idx,cons.conf.idx,y,pcall,age2,young_student,rejecter,old_retired,is_mon_fri
0,56,1,0,0,0,0,1,9.529957,1,0,...,1.1,93.994,-36.4,0,0,256,0,0,0,1
1,57,1,1,1,0,0,1,9.529957,1,0,...,1.1,93.994,-36.4,0,0,289,0,1,0,1
2,37,1,1,0,1,0,1,9.529957,1,0,...,1.1,93.994,-36.4,0,0,9,0,1,0,1
3,40,1,0,0,0,0,1,9.529957,1,0,...,1.1,93.994,-36.4,0,0,0,0,0,0,1
4,56,1,1,0,0,1,1,9.529957,1,0,...,1.1,93.994,-36.4,0,0,256,0,1,0,1


# Split + Scale + Fixing Class Imbalance

In [5]:
X=df.drop('y', axis=1)
y=pd.DataFrame(df.y)

In [6]:
cols=X.columns

In [7]:
X_train, X_test1, y_train, y_test1=train_test_split(X,y,test_size=.3)

In [8]:
mms= MinMaxScaler()

In [9]:
X_train=pd.DataFrame(mms.fit_transform(X_train), columns=cols)

In [10]:
X_test1=pd.DataFrame(mms.transform(X_test1), columns=cols)

In [11]:
ys=np.array(y_train['y'])

In [12]:
X_train['y']=ys

In [13]:
X_train1=X_train[X_train.y>.5]

In [14]:
X_train2= X_train[X_train.y<.5].sample(len(X_train1))

In [15]:
X_train=pd.concat([X_train1, X_train2])

In [16]:
X_train['random']=random.random()
X_train=X_train.sort_values(by='random')
del X_train['random']

In [17]:
y_train=X_train['y']

In [18]:
del X_train['y']

# Lasso Logistic

In [19]:
logistic = LogisticRegression()

In [20]:
penalty = ['l1']

# Create regularization hyperparameter space
#After trying many values for C, the search seems to prefer 10**-0.5
C = C_param_range = [10**-2, 10**-1.5, 10**-1, 10**-.5, 10**0]

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

In [21]:
clf_gs = GridSearchCV(logistic, hyperparameters, cv=20, scoring='f1') 

In [22]:
logreg_model = clf_gs.fit(X_train, y_train)

In [23]:
print('Best Penalty:', logreg_model.best_estimator_.get_params()['penalty'])
print('Best C:', logreg_model.best_estimator_.get_params()['C'])

Best Penalty: l1
Best C: 0.03162277660168379


In [24]:
y_pred_train = logreg_model.best_estimator_.predict(X_train)
f1_score(y_train, y_pred_train)

0.7169633013072925

In [25]:
logreg_model_params = list(zip(cols, logreg_model.best_estimator_.coef_[0]))

logreg_model_params.sort(key = lambda x: abs(x[1]), reverse=True)
logreg_model_params

[('emp.var.rate', -2.2463267326754774),
 ('pdays', 1.3378779435718988),
 ('cons.price.idx', 1.0761313484260866),
 ('month', -0.5740742891619334),
 ('cons.conf.idx', 0.4809931983364659),
 ('contact', -0.3749351210239886),
 ('education', 0.15057126864625278),
 ('default', -0.08753886241979679),
 ('old_retired', 0.07907425691381706),
 ('marital', -0.044303641339170315),
 ('rejecter', -0.03703765385312757),
 ('is_mon_fri', -0.014420499309134074),
 ('age', 0.0),
 ('housing', 0.0),
 ('loan', 0.0),
 ('campaign', 0.0),
 ('previous', 0.0),
 ('poutcome', 0.0),
 ('pcall', 0.0),
 ('age2', 0.0),
 ('young_student', 0.0)]

# Ridge Logistic

In [26]:
logistic2 = LogisticRegression()

In [27]:
penalty = ['l2']


C = C_param_range = [10**-1.5,10**-1, 10**-.5, 10**0, 10**.5, 10**1]

hyperparameters = dict(C=C, penalty=penalty)

In [28]:
#Scaling down correlated columns
corr_2= X_train.corr()**2
for col in corr_2:
    X_train[col]=X_train[col]/(X_train[col].max()*((corr_2[col].sum())**.5))
    X_test1[col]= X_test1[col]/(X_test1[col].max()*((corr_2[col].sum())**.5))

In [29]:
clf_gs2 = GridSearchCV(logistic2, hyperparameters, cv=20, scoring='f1') 

In [30]:
logreg_model2 = clf_gs2.fit(X_train, y_train)

In [31]:
print('Best Penalty:', logreg_model2.best_estimator_.get_params()['penalty'])
print('Best C:', logreg_model2.best_estimator_.get_params()['C'])

Best Penalty: l2
Best C: 10


In [32]:
y_pred_train2 = logreg_model2.best_estimator_.predict(X_train)
f1_score(y_train, y_pred_train2)

0.7125256673511292

In [33]:
logreg_model_params2 = list(zip(cols, logreg_model2.best_estimator_.coef_[0]))

logreg_model_params2.sort(key = lambda x: abs(x[1]), reverse=True)
logreg_model_params2

[('emp.var.rate', -5.234594015770283),
 ('age2', 4.613774605109588),
 ('cons.price.idx', 3.947587129275047),
 ('pdays', 2.25470748197909),
 ('month', -1.5717689903877277),
 ('cons.conf.idx', 1.180113646513269),
 ('contact', -0.9922060919737756),
 ('poutcome', 0.9401047926276991),
 ('pcall', -0.6895392105632521),
 ('campaign', -0.44790659917866144),
 ('age', -0.39349620695363285),
 ('education', 0.2858460417824948),
 ('young_student', 0.26341034432388777),
 ('default', -0.18085559493620357),
 ('marital', -0.1711086110911428),
 ('is_mon_fri', -0.14948772817356104),
 ('loan', 0.059899197807209086),
 ('housing', -0.03964946596678406),
 ('rejecter', 0.028664268770592905),
 ('previous', 0.025267287215682194),
 ('old_retired', 0.007158391427853716)]

# KNN

In [34]:
knn=KNeighborsClassifier()

In [35]:
X_test1_knn=X_test1.copy()


In [36]:
for l in logreg_model_params2:
    X_train[l[0]]=X_train[l[0]]*((abs(l[1]))**.5)
    X_test1_knn[l[0]]=X_test1_knn[l[0]]*((abs(l[1]))**.5)

In [37]:
for col in corr_2:
    X_train[col]=X_train[col]/(((corr_2[col].sum())**.25))
    X_test1_knn[col]=X_test1_knn[col]/(((corr_2[col].sum())**.25))

In [38]:
neighbors= [5,7,9,11,15, 17]
grid_params_KNN = [{'n_neighbors':neighbors, 'weights': [ 'uniform']}] 


In [39]:
gs_KNN_poly = GridSearchCV(knn, grid_params_KNN, cv=5, scoring='f1') 

In [40]:
gs_KNN_poly.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'n_neighbors': [5, 7, 9, 11, 15, 17], 'weights': ['uniform']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [41]:
# Best params
print('Best params: %s' % gs_KNN_poly.best_params_)
# Best training data accuracy
# Predict on test data with best params
y_pred_KNN = gs_KNN_poly.predict(X_train)
# Test data accuracy of model with best par
print('Train set F1 score for best params: %.3f ' % f1_score(y_train, y_pred_KNN))

Best params: {'n_neighbors': 15, 'weights': 'uniform'}
Train set F1 score for best params: 0.741 


# Decision Trees

In [42]:
dt= DecisionTreeClassifier()


In [43]:
#Bringing the data back to regular scale
for col in corr_2:
    X_train[col]=X_train[col]/(X_train[col].max()*((corr_2[col].sum())**.5))
    X_test1[col]= X_test1[col]/(X_test1[col].max()*((corr_2[col].sum())**.5))

In [44]:
parameters={ 'max_depth': [9,12, 15], 'criterion': ['entropy', 'gini'],
            'min_samples_leaf':[15,20,25,35],'min_impurity_decrease':[.001,.0005,0.0003] }

In [45]:
clf_tree=DecisionTreeClassifier()
gs_dt = GridSearchCV(DecisionTreeClassifier(),
                  param_grid= parameters,
                  scoring='f1', cv=5, return_train_score=True)
gs_dt.fit(X_train, y_train)
results = gs_dt.cv_results_

In [46]:
print(gs_dt.best_params_)


{'criterion': 'entropy', 'max_depth': 12, 'min_impurity_decrease': 0.0003, 'min_samples_leaf': 35}


In [47]:
#Decision Tree has highest f1 score
y_pred_dt = gs_dt.best_estimator_.predict(X_train)

print("f1:",f1_score(y_train, y_pred_dt))

f1: 0.7161858697529798


# Random Forest

In [48]:
rfc=RandomForestClassifier()

In [49]:
param_grid = { 
    'n_estimators': [500],
    'max_features': [0.4, 0.5, .6],
    'max_depth' : [10, 12, 14],
    'min_samples_leaf': [0.04,0.05],
    'criterion': [ 'entropy']
}

In [50]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5,n_jobs=-1)
CV_rfc.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [500], 'max_features': [0.4, 0.5, 0.6], 'max_depth': [10, 12, 14], 'min_samples_leaf': [0.04, 0.05], 'criterion': ['entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [51]:
CV_rfc.best_params_

{'criterion': 'entropy',
 'max_depth': 14,
 'max_features': 0.6,
 'min_samples_leaf': 0.04,
 'n_estimators': 500}

In [52]:
rfc_pred = CV_rfc.predict(X_train)


In [53]:
print('Test F1 score: ', f1_score(y_train, rfc_pred))

Test F1 score:  0.6998602375960867


# XG Boost

In [54]:
clf= xgb.XGBClassifier()

In [55]:
param_grid = {
    "learning_rate": [0.1],
    'max_depth': [3],
    'min_child_weight': [40],
    'subsample': [.8],
    'n_estimators': [500]
}

In [56]:
grid_clf = GridSearchCV(clf, param_grid, scoring='f1', cv=5, n_jobs=1)
grid_clf.fit(X_train, y_train)


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'learning_rate': [0.1], 'max_depth': [3], 'min_child_weight': [40], 'subsample': [0.8], 'n_estimators': [500]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [57]:
best_parameters_xg = grid_clf.best_params_
print("Grid Search found the following optimal parameters: ")
for param_name in sorted(best_parameters_xg.keys()):
    print("%s: %r" % (param_name, best_parameters_xg[param_name]))

training_preds = grid_clf.predict(X_train)

Grid Search found the following optimal parameters: 
learning_rate: 0.1
max_depth: 3
min_child_weight: 40
n_estimators: 500
subsample: 0.8


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [58]:
training_accuracy = f1_score(y_train, training_preds)
training_accuracy

0.742007180714652

# AdaBoost

In [59]:
ada= AdaBoostClassifier()

In [60]:
param_grid = {
    'base_estimator': [DecisionTreeClassifier(max_depth=1)],
    'n_estimators': [1000],
    'learning_rate': [.05,.1,.3]
}

In [61]:
grid_ada = GridSearchCV(ada, param_grid, scoring='f1', cv=5, n_jobs=1)
grid_ada.fit(X_train, y_train)


GridSearchCV(cv=5, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'base_estimator': [DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')], 'n_estimators': [1000], 'learning_rate': [0.05, 0.1, 0.3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [62]:
best_parameters_ada = grid_ada.best_params_
print("Grid Search found the following optimal parameters: ")
for param_name in sorted(best_parameters_ada.keys()):
    print("%s: %r" % (param_name, best_parameters_ada[param_name]))

training_preds = grid_ada.predict(X_train)

Grid Search found the following optimal parameters: 
base_estimator: DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
learning_rate: 0.1
n_estimators: 1000


In [63]:
training_accuracy = f1_score(y_train, training_preds)
training_accuracy

0.7126516464471405

# SVM RBF

In [64]:
rbf=svm.SVC()

In [65]:
C_range =  np.array([0.1, 1, 10])  
gamma_range =  np.array([10**-1,10**-2, 10**-3])  
param_grid = dict(gamma=gamma_range, C=C_range)

In [66]:
grid_rbf = GridSearchCV(rbf, param_grid, scoring='f1', cv=None, n_jobs=-1)
grid_rbf.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'gamma': array([0.1  , 0.01 , 0.001]), 'C': array([ 0.1,  1. , 10. ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [67]:
best_parameters_rbf = grid_rbf.best_params_
print("Grid Search found the following optimal parameters: ")
for param_name in sorted(best_parameters_rbf.keys()):
    print("%s: %r" % (param_name, best_parameters_rbf[param_name]))

training_preds = grid_rbf.predict(X_train)

Grid Search found the following optimal parameters: 
C: 1.0
gamma: 0.1


In [68]:
training_accuracy = f1_score(y_train, training_preds)
training_accuracy

0.7182693932736185

# SVM Poly

In [69]:
poly= svm.SVC(kernel='poly')

In [70]:
r_range =  np.array([10**-3,10**-4])  
gamma_range =  np.array([.2]) 
d_range = np.array([2])
param_grid = dict(gamma=gamma_range, degree = d_range, coef0 = r_range)

In [71]:
grid_poly = GridSearchCV(poly, param_grid, scoring='f1', cv=None, n_jobs=-1)
grid_poly.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'gamma': array([0.2]), 'degree': array([2]), 'coef0': array([0.001 , 0.0001])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [72]:
best_parameters_poly = grid_poly.best_params_
print("Grid Search found the following optimal parameters: ")
for param_name in sorted(best_parameters_poly.keys()):
    print("%s: %r" % (param_name, best_parameters_poly[param_name]))

training_preds = grid_poly.predict(X_train)

Grid Search found the following optimal parameters: 
coef0: 0.001
degree: 2
gamma: 0.2


In [73]:
training_accuracy = f1_score(y_train, training_preds)
training_accuracy

0.7218444186432245

# SVM Sigmoid

In [74]:
sig=svm.SVC(kernel='sigmoid')

In [75]:
r_range =  np.array([0.001, 0.01, .1])  
gamma_range =  np.array([0.001, 0.01, 0.1]) 
param_grid = dict(gamma=gamma_range,coef0 = r_range)

In [76]:
grid_sig = GridSearchCV(sig, param_grid, scoring='f1', cv=None, n_jobs=-1)
grid_sig.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='sigmoid',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'gamma': array([0.001, 0.01 , 0.1  ]), 'coef0': array([0.001, 0.01 , 0.1  ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [77]:
best_parameters_sig = grid_sig.best_params_
print("Grid Search found the following optimal parameters: ")
for param_name in sorted(best_parameters_sig.keys()):
    print("%s: %r" % (param_name, best_parameters_sig[param_name]))

training_preds = grid_sig.predict(X_train)

Grid Search found the following optimal parameters: 
coef0: 0.1
gamma: 0.1


In [78]:
training_accuracy = f1_score(y_train, training_preds)
training_accuracy

0.7025243359181653

# Val-Train Split

In [161]:
X_val, X_test, y_val, y_test= train_test_split(X_test1, y_test1, test_size=.5)

In [162]:
X_val_k=X_val.copy()
X_test_k=X_test.copy()

In [163]:
for l in logreg_model_params2:
    X_val_k[l[0]]=X_val_k[l[0]]*((abs(l[1]))**.5)
    X_test_k[l[0]]=X_test_k[l[0]]*((abs(l[1]))**.5)

In [164]:
for col in corr_2:
    X_val_k[col]=X_val_k[col]/(((corr_2[col].sum())**.25))
    X_test_k[col]=X_test_k[col]/(((corr_2[col].sum())**.25))

# Validation

In [165]:
y_val_log2 = logreg_model2.best_estimator_.predict(X_val)
f1_score(y_val, y_val_log2)

0.44885177453027136

In [166]:
y_val_knn = gs_KNN_poly.predict(X_val)
f1_score(y_val, y_val_knn)

0.29865985960433955

In [167]:
y_val_dt = gs_dt.best_estimator_.predict(X_val)
f1_score(y_val, y_val_dt)

0.4287886467308667

In [168]:
y_val_rf= rfc_pred = CV_rfc.predict(X_val)
f1_score(y_val, y_val_rf)

0.45179063360881544

In [169]:
y_val_ada= grid_ada.predict(X_val)
f1_score(y_val, y_val_ada)

0.4657236126224157

In [170]:
y_val_xg = grid_clf.predict(X_val)
f1_score(y_val, y_val_xg)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.44992295839753466

In [171]:
y_val_rbf= grid_rbf.predict(X_val)
f1_score(y_val, y_val_rbf)

0.442663995993991

In [172]:
y_val_poly= grid_poly.predict(X_val)
f1_score(y_val, y_val_poly)

0.4221908791725435

In [173]:
y_val_sig= grid_sig.predict(X_val)
f1_score(y_val, y_val_sig)

0.3901565995525727

# Voting Ensemble

In [174]:
y_df= y_val.copy()

In [175]:
y_df['log2']= y_val_log2
y_df['knn']=y_val_knn
y_df['dt']=y_val_dt
y_df['rf']=y_val_rf
y_df['xg']=y_val_xg
y_df['rbf']=y_val_rbf
y_df['ada']=y_val_ada

In [176]:
models=y_df.drop(columns='y')

In [177]:
models=models-.5

In [178]:
models.describe()

,log2,knn,dt,rf,xg,rbf,ada
count,6175.000000,6175.000000,6175.000000,6175.000000,6175.000000,6175.000000,6175.000000
mean,-0.298704,-0.101457,-0.289474,-0.315061,-0.293684,-0.285587,-0.311336
std,0.401001,0.489638,0.407715,0.388280,0.404693,0.410448,0.391273
min,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000
25%,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000
50%,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000
75%,-0.500000,0.500000,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000
max,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000


In [179]:
models['vote']=0
for col in models.columns[:-1]:
    models['vote']= models['vote']+ models[col]


In [180]:
models['vote2']=0
for col in ['log2', 'rbf', 'rf', 'xg', 'ada']:
    models['vote2']= models['vote2']+ models[col]

In [181]:
models['vote3']=0
for col in ['log2',  'rf', 'ada']:
    models['vote3']= models['vote3']+ models[col]

In [182]:
models['y_guess_1']= (np.sign(models.vote)+1)/2
models['y_guess_2']= (np.sign(models.vote2)+1)/2
models['y_guess_3']= (np.sign(models.vote3)+1)/2

In [183]:
f1_score(y_val['y'], np.array(models.y_guess_1))

0.46612466124661245

In [184]:
f1_score(np.array(y_val['y']), np.array(models.y_guess_2))

0.46046261430876817

In [185]:
f1_score(np.array(y_val['y']), np.array(models.y_guess_3))

0.46153846153846145

# Ada Ensemble

In [186]:
model_ada= AdaBoostClassifier()

In [187]:
param_grid = {
    'base_estimator': [DecisionTreeClassifier(max_depth=1)],
    'n_estimators': [1000],
    'learning_rate': [.05,.1,.3]
}

In [188]:
ada_y= y_df.copy()

In [189]:
ada_y_t= ada_y[ada_y['y']>.5]
ada_y_f= ada_y[ada_y['y']<.5].sample(len(ada_y_t))

In [190]:
ada_y= pd.concat([ada_y_t, ada_y_f])

In [191]:
ada_y['random']=random.random()
ada_y.sort_values(by='random')
del ada_y['random']

In [192]:
ada=ada_y.drop(columns='y')

In [193]:
ada_y= ada_y['y']

In [194]:
grid_model_ada = GridSearchCV(model_ada, param_grid, scoring='f1', cv=5, n_jobs=1)
grid_model_ada.fit(ada, ada_y)

GridSearchCV(cv=5, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'base_estimator': [DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')], 'n_estimators': [1000], 'learning_rate': [0.05, 0.1, 0.3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [195]:
best_parameters_model_ada = grid_model_ada.best_params_
print("Grid Search found the following optimal parameters: ")
for param_name in sorted(best_parameters_model_ada.keys()):
    print("%s: %r" % (param_name, best_parameters_model_ada[param_name]))

training_preds = grid_model_ada.predict(ada)

Grid Search found the following optimal parameters: 
base_estimator: DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
learning_rate: 0.05
n_estimators: 1000


In [196]:
training_accuracy = f1_score(ada_y, training_preds)
training_accuracy

0.726825266611977

# Log Reg Ensemble

In [197]:
lr_model= LogisticRegression()

In [198]:
param_grid = {
    'penalty': ['l2'],
    'C': [10**-3, 10**0, 10**3, 10**6]
    
}

In [199]:
grid_model_lr = GridSearchCV(lr_model, param_grid, scoring='f1', cv=5, n_jobs=1)
grid_model_lr.fit(ada, ada_y)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l2'], 'C': [0.001, 1, 1000, 1000000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [200]:
best_parameters_model_lr = grid_model_lr.best_params_
print("Grid Search found the following optimal parameters: ")
for param_name in sorted(best_parameters_model_lr.keys()):
    print("%s: %r" % (param_name, best_parameters_model_lr[param_name]))

training_preds = grid_model_lr.predict(ada)

Grid Search found the following optimal parameters: 
C: 1
penalty: 'l2'


In [201]:
training_accuracy = f1_score(ada_y, training_preds)
training_accuracy

0.7293540474243663

# Testing 

In [202]:
y_test_log2 = logreg_model2.best_estimator_.predict(X_test)
f1_score(y_test, y_test_log2)

0.43582089552238806

In [203]:
y_test_knn = gs_KNN_poly.predict(X_test)
f1_score(y_test, y_test_knn)

0.3109004739336493

In [204]:
y_test_dt = gs_dt.best_estimator_.predict(X_test)
f1_score(y_test, y_test_dt)

0.4122590212555611

In [205]:
y_test_rf= rfc_pred = CV_rfc.predict(X_test)
f1_score(y_test, y_test_rf)

0.4451544195953141

In [206]:
y_test_ada= grid_ada.predict(X_test)
f1_score(y_test, y_test_ada)

0.4509394572025052

In [207]:
y_test_xg = grid_clf.predict(X_test)
f1_score(y_test, y_test_xg)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.4323790720631787

In [208]:
y_test_rbf= grid_rbf.predict(X_test)
f1_score(y_test, y_test_rbf)

0.4247957712638155

# Testing Ensemble

In [209]:
y_test_df= y_test.copy()

In [210]:
y_test_df['log2']= y_test_log2
y_test_df['knn']=y_test_knn
y_test_df['dt']=y_test_dt
y_test_df['rf']=y_test_rf
y_test_df['xg']=y_test_xg
y_test_df['rbf']=y_test_rbf
y_test_df['ada']=y_test_ada

In [211]:
del y_test_df['y']

In [213]:
y_test_df+= (-.5)

In [216]:
y_test_df['vote']=0
for col in y_test_df.columns[:-1]:
    y_test_df['vote']= y_test_df['vote']+ y_test_df[col]

In [217]:
y_test_df['y_guess']= (np.sign(y_test_df.vote)+1)/2


In [219]:
f1_score(y_test, y_test_df['y_guess'])

0.4489372731985485